# Final Project

## Notes

In [1]:
'''
Using the following machine learning models for making predictions for the following portfolio optimization models:
- Mean-variance optimization (chapter 7 of Bodie, Investment book)
- Index model (chapter 8 of Bodie, Investment book)
- Capital asset pricing model (chapter 9 of Bodie, Investment book)
- Arbitrage pricing theory and multifactor model (chapter 10 of Bodie,
Investment book)
- Equity valuation model (chapter 18 of Bodie, Investment book)
- Black Litterman model (chapter 24 of Bodie, Investment book)
- Algorithmic trading (this could be restricted to the last month)
'''

'\nUsing the following machine learning models for making predictions for the following portfolio optimization models:\n- Mean-variance optimization (chapter 7 of Bodie, Investment book)\n- Index model (chapter 8 of Bodie, Investment book)\n- Capital asset pricing model (chapter 9 of Bodie, Investment book)\n- Arbitrage pricing theory and multifactor model (chapter 10 of Bodie,\nInvestment book)\n- Equity valuation model (chapter 18 of Bodie, Investment book)\n- Black Litterman model (chapter 24 of Bodie, Investment book)\n- Algorithmic trading (this could be restricted to the last month)\n'

## Load Data

In [2]:
import yfinance as yf
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from pypfopt import expected_returns, risk_models, EfficientFrontier

tickers = ["AAPL", "AMZN", "BRK-B", "GOOGL", "META", "MSFT", "NVDA", "TSLA", "UNH", "XOM"]
portfolio_data = yf.download(tickers, period="5y")
portfolio_data = portfolio_data["Adj Close"]
portfolio_data.tail()

[*********************100%***********************]  10 of 10 completed


,AAPL,AMZN,BRK-B,GOOGL,META,MSFT,NVDA,TSLA,UNH,XOM
Date,,,,,,,,,,
2023-06-26,185.270004,127.330002,334.119995,118.339996,278.470001,328.600006,406.320007,241.050003,479.059998,104.290001
2023-06-27,188.059998,129.179993,335.339996,118.330002,287.049988,334.570007,418.760010,250.210007,482.559998,104.550003
2023-06-28,189.250000,129.039993,334.149994,120.180000,285.290009,335.850006,411.170013,256.239990,474.450012,105.400002
2023-06-29,189.589996,127.900002,336.910004,119.099998,281.529999,335.049988,408.220001,257.500000,476.440002,106.699997
2023-06-30,193.970001,130.360001,341.000000,119.699997,286.980011,340.540009,423.019989,261.769989,480.640015,107.250000


In [3]:
market_index_ticker = "^GSPC"  # S&P 500 index
market_index_data = yf.download(market_index_ticker, period="5y")
market_index_data = market_index_data["Adj Close"]
market_index_data

[*********************100%***********************]  1 of 1 completed


Date
2018-07-02    2726.709961
2018-07-03    2713.219971
2018-07-05    2736.610107
2018-07-06    2759.820068
2018-07-09    2784.169922
                 ...     
2023-06-26    4328.819824
2023-06-27    4378.410156
2023-06-28    4376.859863
2023-06-29    4396.439941
2023-06-30    4450.379883
Name: Adj Close, Length: 1258, dtype: float64

## Mean Variance Optimization

### Random Forest 

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    # Dependent variable - 10 consecutive days of stock prices
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    
    # Independent variable - stock price 10th day into the future
    dev_y = [data[i+10] for i in range(10,len(data)-10)]
    
    test = [data[i:i+10] for i in range(len(data)-20,len(data)-10)]
    
    reg = RandomForestRegressor()
    reg.fit(dev_x,dev_y)
    
    # Predict stock price for 10 future days
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)
# Construct covariance matrix of future stock prices
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)

# Do mean variance optimization using efficient frontier
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=0.02)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### SVM

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = SVR()
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)


ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### Neural Network

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = MLPRegressor(hidden_layer_sizes=(100,100), random_state=42)  # You can adjust the hidden_layer_sizes and other parameters as needed
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)



## Index model

### Random Forest for predicting market index values

In [11]:
# Returns of stocks and market
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    
# Future marker returns prediction
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]

market_test = [returns["MARKET"][i:i+10]
              for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = RandomForestRegressor()
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)
market_future

array([-0.00118954,  0.00018475,  0.00040524,  0.00160235,  0.00235952,
        0.00090033,  0.00140159, -0.00033955, -0.0006818 , -0.00048814])

In [12]:
# For each stock in the portfolio,
# Fit a single index model regression on market return and stock return
# Determine alpha, beta, variance of the residuals

beta = {}
alpha = {}
residual_variance = {}

for ticker in tickers:
    single_index_reg = LinearRegression()
    single_index_reg.fit(np.array(returns["MARKET"]).reshape(-1, 1),y=returns[ticker])
    stock_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

    beta[ticker] = single_index_reg.coef_[0]
    alpha[ticker] = single_index_reg.intercept_

    y_pred = single_index_reg.predict(np.array(returns["MARKET"]).reshape(-1, 1))
    residuals = returns[ticker] - y_pred
    residual_variance[ticker] = np.var(residuals)

In [13]:
# Using Treynor Black model for portfolio optimization

# STEP 1:
# Compute the initial position of each security:
weights = {ticker: alpha[ticker] / residual_variance[ticker] for ticker in tickers}
print(f"Weights: {weights}\n")

# STEP 2:
# Scale the initial positions:
total_weight = sum(weights.values())
weights = {ticker: weight / total_weight for ticker, weight in weights.items()}
print(f"Scaled weights: {weights}\n")

# STEP 3:
# Compute the alpha of the active portfolio:
alpha_portfolio = sum(weights[ticker] * alpha[ticker] for ticker in tickers)
print(f"Alpha of the active portfolio: {alpha_portfolio}\n")

# STEP 4:
# Compute the residual variance of the active portfolio:
residual_variance_portfolio = sum((weights[ticker] ** 2) * residual_variance[ticker] for ticker in tickers)
print(f"Residual variance of the active portfolio: {residual_variance_portfolio}\n")

# STEP 5:
# Compute the initial position in the active portfolio:
residual_variance_market = 0.0114  # Variance of S&P 500
risk_premium_market = 0.056
initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)
print(f"Initial position of the active portfolio: {initial_position_portfolio}\n")

# STEP 6:
# Compute the beta of the active portfolio:
beta_portfolio = sum(weights[ticker] * beta[ticker] for ticker in tickers)
print(f"Beta of the active portfolio: {beta_portfolio}\n")

#STEP 7:
# Adjust the initial position in active portfolio
adjusted_position_portfolio = initial_position_portfolio/1+(1-beta_portfolio)*initial_position_portfolio
print(f"Adjusted position of the active portfolio: {adjusted_position_portfolio}")

# STEP 8:
# Optimal risky portfolio now has weights:
final_weight_market = 1 - adjusted_position_portfolio
weights = {ticker: weight * adjusted_position_portfolio for ticker, weight in weights.items()}

print("------------------------------")
print("Final Weights:")
print("Weight Market S&P:", final_weight_market)
print(f"Weights: {weights}")
print("------------------------------")

#STEP 9:
# Calculate the risk premium of P (Optimal risky portfolio):
risk_premium_porfolio = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*risk_premium_market + adjusted_position_portfolio*alpha_portfolio
print("Risk premium of portfolio: ", risk_premium_porfolio)

#STEP 10:
# Compute the variance of Portfolio:
portfolio_variance = (final_weight_market+adjusted_position_portfolio*beta_portfolio)*(final_weight_market+adjusted_position_portfolio*beta_portfolio)*residual_variance_market + adjusted_position_portfolio*adjusted_position_portfolio*residual_variance_portfolio
print("Variance of portfolio: ", portfolio_variance)

sharpe_ratio = risk_premium_porfolio / (portfolio_variance**0.5)
print(f"Sharpe ratio: {sharpe_ratio}")

Weights: {'AAPL': 5.122452599826317, 'AMZN': 0.19608491512600934, 'BRK-B': 2.4339195401637594, 'GOOGL': 1.4863579468455903, 'META': 0.17266094642367638, 'MSFT': 5.505126547305852, 'NVDA': 2.240093064225395, 'TSLA': 1.5722516686676098, 'UNH': 1.6361742826957755, 'XOM': 0.7010626883133609}

Scaled weights: {'AAPL': 0.24315996439095022, 'AMZN': 0.009308041421654069, 'BRK-B': 0.11553680140187622, 'GOOGL': 0.07055658171232938, 'META': 0.008196118707962753, 'MSFT': 0.26132528298181884, 'NVDA': 0.10633596682728318, 'TSLA': 0.07463390872173044, 'UNH': 0.07766827951344693, 'XOM': 0.03327905432094788}

Alpha of the active portfolio: 0.0007219367631504806

Residual variance of the active portfolio: 3.426993499679057e-05

Initial position of the active portfolio: 4.288473212060074

Beta of the active portfolio: 1.2086189451637475

Adjusted position of the active portfolio: 3.393816454197114
------------------------------
Final Weights:
Weight Market S&P: -2.393816454197114
Weights: {'AAPL': 0.8252

### Capital asset pricing model (CAPM)

### Arbitrage pricing theory and multifactor model

### Equity valuation model

### Black Litterman model 

### Algorithmic trading

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Create random input data
X = np.random.rand(100, 10)  # 100 samples with 10 features each
y = np.random.randint(2, size=(100, 1))  # Binary target variable

# Define the neural network architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=10))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X)


In [ ]:
import numpy as np

# Historical stock returns
aapl_returns = np.array([0.02, 0.05, -0.01, 0.03, 0.01])  # Replace with actual returns
msft_returns = np.array([0.03, 0.04, 0.01, -0.02, 0.02])  # Replace with actual returns
amzn_returns = np.array([0.04, -0.01, 0.02, 0.03, 0.05])  # Replace with actual returns

# Portfolio weights
weights = np.array([0.4, 0.3, 0.3])  # Adjust the weights as per your desired allocation

# Calculate portfolio returns
portfolio_returns = np.dot(np.array([aapl_returns, msft_returns, amzn_returns]).T, weights)

# Calculate portfolio standard deviation
portfolio_std = np.std(portfolio_returns)

# Risk-free rate (assumed to be 2%)
risk_free_rate = 0.02

# Calculate Sharpe ratio
sharpe_ratio = (np.mean(portfolio_returns) - risk_free_rate) / portfolio_std

print("Sharpe Ratio:", sharpe_ratio)


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Historical asset returns
aapl_returns = np.array([0.02, 0.05, -0.01, 0.03, 0.01])  # Replace with actual returns
msft_returns = np.array([0.03, 0.04, 0.01, -0.02, 0.02])  # Replace with actual returns
amzn_returns = np.array([0.04, -0.01, 0.02, 0.03, 0.05])  # Replace with actual returns

# Historical market returns
market_returns = np.array([0.02, 0.03, 0.01, 0.02, 0.04])  # Replace with actual returns

# Perform linear regression to estimate asset betas
regressor = LinearRegression()
regressor.fit(market_returns.reshape(-1, 1), aapl_returns)
aapl_beta = regressor.coef_[0]

regressor.fit(market_returns.reshape(-1, 1), msft_returns)
msft_beta = regressor.coef_[0]

regressor.fit(market_returns.reshape(-1, 1), amzn_returns)
amzn_beta = regressor.coef_[0]

# Determine the expected market return and risk-free rate
expected_market_return = np.mean(market_returns)
risk_free_rate = 0.02

# Set the range of weights to try
min_weight = 0
max_weight = 1
step_size = 0.1

best_sharpe_ratio = -np.inf
optimal_weights = None

# Iterate over different weight combinations
for aapl_weight in np.arange(min_weight, max_weight + step_size, step_size):
    for msft_weight in np.arange(min_weight, max_weight - aapl_weight + step_size, step_size):
        amzn_weight = 1 - aapl_weight - msft_weight

        # Calculate expected returns using the single index model formula
        aapl_expected_return = risk_free_rate + aapl_beta * (expected_market_return - risk_free_rate)
        msft_expected_return = risk_free_rate + msft_beta * (expected_market_return - risk_free_rate)
        amzn_expected_return = risk_free_rate + amzn_beta * (expected_market_return - risk_free_rate)

        # Calculate portfolio expected return
        portfolio_expected_return = aapl_weight * aapl_expected_return + msft_weight * msft_expected_return + amzn_weight * amzn_expected_return

        # Calculate portfolio risk (variance)
        asset_covariance = np.cov(np.array([aapl_returns, msft_returns, amzn_returns]))
        portfolio_variance = (aapl_weight ** 2) * asset_covariance[0, 0] + (msft_weight ** 2) * asset_covariance[1, 1] + (amzn_weight ** 2) * asset_covariance[2, 2] + 2 * aapl_weight * msft_weight * asset_covariance[0, 1] + 2 * aapl_weight * amzn_weight * asset_covariance[0, 2] + 2 * msft_weight * amzn_weight * asset_covariance[1, 2]

        # Calculate Sharpe ratio
        sharpe_ratio = (portfolio_expected_return - risk_free_rate) / np.sqrt(portfolio_variance)

        # Update best Sharpe ratio and optimal weights if a higher value is found
        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            optimal_weights = (aapl_weight, msft_weight, amzn_weight)

print("Optimal Weights:", optimal_weights)
print("Best Sharpe Ratio:", best_sharpe_ratio)


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
tickers = ["MSFT", "AMZN", "KO", "MA", "COST", 
           "LUV", "XOM", "PFE", "JPM", "UNH", 
           "ACN", "DIS", "GILD", "F", "TSLA"] 
ohlc = yf.download(tickers, period="5y")

In [ ]:
prices = ohlc["Adj Close"].dropna(how="all")
prices.tail()